In [1]:
import os
os.environ['PYKX_4_1_ENABLED'] = "True"
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


## --- Day 10: Factory --- [link](https://adventofcode.com/2025/day/10)
Just across the hall, you find a large factory. Fortunately, the Elves here have plenty of time to decorate. Unfortunately, it's because the factory machines are all offline, and none of the Elves can figure out the initialization procedure.

The Elves do have the manual for the machines, but the section detailing the initialization procedure was eaten by a Shiba Inu. All that remains of the manual are some indicator light diagrams, button wiring schematics, and joltage requirements for each machine.

For example:

    [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
    [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
    [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}


In [11]:
show lines:read0`:test.txt

"[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}"
"[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}"
"[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"


In [12]:
/ use filter function to pre-split by space - convenient!
show .Q.s1 each input:{[x:vs[" "]] ("#"=-1_1_x 0;get each -1_1_x;get -1_1_last x)} each lines

"(0110b;(3;1 3;2;2 3;0 2;0 1);3 5 4 7)"
"(00010b;(0 2 3 4;2 3;0 4;0 1 2;1 2 3 4);7 5 12 7 2)"
"(011101b;(0 1 2 3 4;0 3 4;0 1 2 4 5;1 2);10 11 11 5 10 5)"


In [13]:
/ flip
@[0000b;1 2;:;not 0000b 1 2]
flp:{@[x;y;:;not x y]}

0110b


In [14]:
/ apply all button flips and increment counter
{[b;(m;c)](distinct raze m flp\:/:b;c+1)}[input . 0 1;(enlist count[input . 0 0]#0b;0)]

(0001b;0101b;0010b;0011b;1010b;1100b)
1


In [15]:
/ use "do-while" iterator
{not 0110b in x 0}{[b;(m;c)] (distinct raze m flp\:/:b;c+1)}[(3;1 3;2;2 3;0 2;0 1)]/(enlist 0000b;0)

(0000b;0100b;0011b;0010b;1011b;1101b;0111b;0110b;1111b;1001b;0001b;1000b;1110b)
2


In [16]:
fn1:{[(tgt;b)] {not x in y 0}[tgt]{[b;(m;c)](distinct raze m flp\:/:b;c+1)}[b]/(enlist count[tgt]#0b;0)}
fn1 -1_input 0

(0000b;0100b;0011b;0010b;1011b;1101b;0111b;0110b;1111b;1001b;0001b;1000b;1110b)
2


In [17]:
sum 0N!last each fn1 each input[;0 1]

72 3 2



In [18]:
/ part 2 - try to use memo
show input2:1_'input

(3;1 3;2;2 3;0 2;0 1)           3 5 4 7                      
0 2 3 4 2 3 0 4 0 1 2 1 2 3 4   7       5   12  7     2      
(0 1 2 3 4;0 3 4;0 1 2 4 5;1 2) 10 11 11 5 10 5              


In [19]:
/ prepare memo
(b;tgt):input2 0
d:(enlist count[tgt]#0)!enlist 0
d

/ try incrementing with buttons
show res:@[count[tgt]#0;;+;1]'[b];

0 0 0 0| 00 0 0 1
0 1 0 1
0 0 1 0
0 0 1 1
1 0 1 0
1 1 0 0



In [20]:
/ now use min path length
d[(0 0 0 0;0 1 1 2)]:10 1
d[res]:1+d[0 0 0 0]&0W^d[res]
show d;

0 0 0 0| 10
0 1 1 2| 1
0 0 0 1| 11
0 1 0 1| 11
0 0 1 0| 11
0 0 1 1| 11
1 0 1 0| 11
1 1 0 0| 11


In [21]:
/ combine
\T 5
(b;tgt):input2 0
d:(start:enlist count[tgt]#0)!enlist 0
part2:{not x in key y 1}[tgt] {[b;(c;d)]
    res:(raze c @[;;+;1]\:/:b) except c;
    d[res]:1+raze count[res]#d[c];
    (res;d)
 }[b]/(start;d);
part2[1;tgt]

10


In [30]:
fn2:{[(b;tgt)]
    d:(start:enlist count[tgt]#0)!enlist 0;
    {not x in key y 1}[tgt] {[b;(c;d)]
        res:(raze c @[;;+;1]\:/:b) except c;
        d[res]:1+raze count[res]#d[c];
        (res;d)
     }[b]/(start;d)
 };
part2:fn2 each input2
sum {x[1;y 1]}'[part2;input2] 

In [36]:
/ too slow on real input :(
/ will use MILP from scipy instead

In [21]:
%%py
import numpy as np
from scipy.optimize import milp,LinearConstraint

filename = 'test.txt'
inputs = []

def parse_line(line : str):
    tokens = line.translate(str.maketrans('(){}', '{}[]')).split(' ')
    target = np.array(eval(tokens[-1]))
    M = []
    for idx in tokens[1:-1]:
        M.append([1 if i in eval(idx) else 0 for i in range(len(target))])

    return (np.array(M).T, target)

with open(filename) as f:
    for line in f:
        inputs.append(parse_line(line))

part2 = 0
for (A,b) in inputs:
    c = np.repeat(1, len(A[0]))
    constraints = LinearConstraint(A, b, b)
    integrality = np.ones_like(c)
    res = milp(c=c, constraints=constraints, integrality=integrality)
    part2 += np.sum(res.x)
    
part2

np.float64(33.0)